In [25]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader 
import transformers
from transformers import GPT2Tokenizer, GPT2Model
from tqdm import tqdm
import os

In [29]:
data = pd.read_feather('../mini_codenet/data/split/finetune_test.ftr')
data.head()

,level_0,index,submission_id,problem_id,language,filename_ext,status,cpu_time,memory,code_size,accuracy,solution,problem_statement
0,2016842,19846,s534157776,p03038,C++,cpp,Compile Error,NaN,NaN,1191,None,//Keep working hard :)\n#include <bits/stdc++....,Score : 400 points \n Problem Statement You ha...
1,485429,3052,s847351877,p02762,C++,cpp,Accepted,130.0,4352.0,2522,None,#include <bits/stdc++.h>\nusing namespace std;...,Score : 400 points \n Problem Statement An SNS...
2,2613417,1022,s183365047,p02555,C++,cpp,Accepted,18.0,3552.0,747,None,#include <bits/stdc++.h>\n// #include <atcoder...,Score : 400 points \n Problem Statement Given ...
3,2277210,4041,s610113411,p03315,Python,py,Accepted,17.0,2940.0,102,None,S=list(input())\nans=0\nfor i in range(len(S))...,Score : 100 points \n Problem Statement There ...
4,2456464,5347,s811377587,p03470,C++,cpp,Accepted,1.0,256.0,290,None,#include <bits/stdc++.h>\nusing namespace std;...,Score : 200 points \n Problem Statement An X -...


In [30]:
data.iloc[0]["solution"]

'//Keep working hard :)\n#include <bits/stdc++.h>\nusing namespace std;\n\n#define IOS ios::sync_with_stdio(0); cin.tie(0); cout.tie(0);\n#define int long long\n\nvoid fileIO()\n{\n        #ifndef ONLINE_JUDGE\n        freopen("int.txt","r",stdin);\n        freopen("out.txt","w",stdout);\n        #endif\n}\nint main()\n{\n\tIOS;\n       // fileIO();\n        multiset<int> s;\n        int n,m;\n        cin >> n >> m;\n        for(int i = 0; i < n; i++)\n        {\n                int x;\n                cin >> x;\n                s.insert(x);\n        }\n        for(int i = 0; i < m; i++)\n        {\n                int b,c;\n                cin >> b >> c;\n                for(int j = 0; j < b; j++)\n                {\n                        auto it = s.begin();\n                        if(*it < c)\n                        {\n                                s.erase(it);\n                                s.insert(c);\n                        }\n                        else\n             

In [31]:
print(set(data['problem_id'].values))

{'p02402', 'p03316', 'p03104', 'p02995', 'p02969', 'p03464', 'p03129', 'p02612', 'p03846', 'p03795', 'p02957', 'p02400', 'p03440', 'p03012', 'p03254', 'p02827', 'p02920', 'p03618', 'p03473', 'p02470', 'p02923', 'p03424', 'p02777', 'p02846', 'p02579', 'p04002', 'p03109', 'p03498', 'p02379', 'p02577', 'p02798', 'p03047', 'p03487', 'p03183', 'p03272', 'p02734', 'p03714', 'p02949', 'p03315', 'p03297', 'p02708', 'p03055', 'p03180', 'p02939', 'p03034', 'p03081', 'p03434', 'p02891', 'p02748', 'p03760', 'p02693', 'p02615', 'p03715', 'p02641', 'p03659', 'p03404', 'p03575', 'p02661', 'p03103', 'p02264', 'p03937', 'p02765', 'p03026', 'p02678', 'p03526', 'p03988', 'p02717', 'p03472', 'p00914', 'p03770', 'p03369', 'p03045', 'p02354', 'p02581', 'p03796', 'p03378', 'p02835', 'p03011', 'p00750', 'p02392', 'p02775', 'p03408', 'p03660', 'p03251', 'p02850', 'p04037', 'p00738', 'p03075', 'p02781', 'p03063', 'p00746', 'p02762', 'p02866', 'p02686', 'p00723', 'p02856', 'p03780', 'p00001', 'p02861', 'p02088',

In [32]:
coll = []
for pid in set(data['problem_id'].values):
    yes = data.loc[(data['status'] == 'Accepted') & (data['language'] == 'Python') & (data['problem_id'] == pid)].values
    coll.append(yes)

In [41]:
print(len(coll))
coll2 = []
for col in coll:
    if len(col) > 0:
        #print(col)
        # print(col[0][3])
        coll2.append(col[0][3])
print(coll2)

1081
['p02402', 'p03316', 'p02995', 'p03129', 'p02612', 'p02400', 'p03618', 'p02923', 'p03424', 'p03109', 'p02577', 'p03714', 'p03315', 'p02939', 'p03434', 'p03760', 'p02693', 'p02641', 'p03575', 'p03103', 'p02765', 'p03026', 'p02678', 'p03472', 'p03045', 'p03796', 'p03378', 'p03011', 'p02392', 'p03075', 'p02781', 'p02856', 'p00001', 'p02861', 'p03339', 'p02860', 'p03013', 'p02410', 'p03370', 'p03102', 'p02887', 'p03479', 'p02404', 'p02981', 'p02726', 'p02646', 'p02950', 'p03220', 'p03164', 'p02675', 'p03852', 'p03502', 'p03624', 'p03372', 'p03171', 'p03394', 'p02886', 'p03644', 'p02700', 'p02851', 'p02628', 'p02682', 'p04029', 'p03987', 'p02803', 'p03087', 'p04043', 'p02912', 'p03776', 'p02657', 'p03107', 'p03970', 'p03241', 'p02837', 'p01102', 'p02389', 'p02619', 'p02836', 'p03455', 'p03448', 'p02729', 'p03547', 'p02694', 'p03351', 'p02681', 'p02262', 'p02759', 'p02598', 'p03359', 'p02711', 'p03501', 'p03186', 'p02608', 'p03814', 'p03317', 'p02871', 'p03265', 'p03525', 'p04033', 'p02

In [4]:
class CodeNetDataset(Dataset):
    def __init__(self, feather_path, tokenizer):
        self.data = pd.read_feather(feather_path)
        if tokenizer.pad_token is None:
            tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        instance = self.data.iloc[idx]
        # language = self.tokenizer(instance["language"], return_tensors='pt')
        # problem = self.tokenizer(instance["language"] + "::" + instance["problem_statement"], max_length=1028, padding='max_length', truncation=True, return_tensors='pt')
        # code_solution = self.tokenizer(instance["solution"], max_length=1028, padding='max_length', truncation=True, return_tensors='pt')

        problem = self.tokenizer(instance["language"] + "::" + instance["problem_statement"], return_tensors='pt')
        code_solution = self.tokenizer(instance["solution"], return_tensors='pt')

        return instance['problem_id'], problem, code_solution

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')
model = GPT2Model.from_pretrained('gpt2-large')

Using pad_token, but it is not set yet.


In [20]:
cnds = CodeNetDataset('../mini_codenet/data/split/pretrain_train.ftr', tokenizer)

In [28]:
n, p, s = cnds[0]
print(n)
print(p.input_ids.shape)
print(s.input_ids.shape)
arr = []
arr.append([n,p.input_ids.shape,s.input_ids.shape])
test_df = pd.DataFrame(arr, columns=['problem_id','input_shape','output_shape'])
test_df.head()

p02936
torch.Size([1, 663])
torch.Size([1, 187])


,problem_id,input_shape,output_shape
0,p02936,"(1, 663)","(1, 187)"


In [11]:
#print(os.listdir('../mini_codenet/data/split/'))

['.gitkeep', 'evaluate_test.ftr', 'evaluate_train.ftr', 'evaluate_val.ftr', 'pretrain_test.ftr', 'pretrain_train.ftr', 'pretrain_val.ftr', 'reward_test.ftr', 'reward_train.ftr', 'reward_val.ftr']


In [29]:
filenames = os.listdir('../mini_codenet/data/split/')
# get all of the encoding lengths for a file
i = 0
for filename in filenames:
    if filename != '.gitkeep':
        cnds = CodeNetDataset(f'../mini_codenet/data/split/{filename}', tokenizer)
        print(i, filename)
        array = []
        #array.append(['problem_id','input_shape','output_shape'])
        for batch in tqdm(cnds):
            n, p, s = batch
            array.append([n, p.input_ids.shape, s.input_ids.shape])
    
        df = pd.DataFrame(array, columns=['problem_id','input_shape','output_shape'])
        print(df.head())
        df.reset_index(inplace=True)
        df.to_feather(f"./encoding_analysis/{filename}")
        i += 1


0 evaluate_test.ftr


100%|██████████| 190378/190378 [15:55<00:00, 199.24it/s]


  problem_id input_shape output_shape
0     p02936    (1, 663)     (1, 187)
1     p02936    (1, 663)     (1, 255)
2     p02936    (1, 663)     (1, 273)
3     p02936    (1, 663)     (1, 340)
4     p02936    (1, 664)     (1, 541)
1 evaluate_train.ftr


100%|██████████| 1332650/1332650 [2:04:54<00:00, 177.81it/s]  


  problem_id input_shape output_shape
0     p02700    (1, 362)     (1, 181)
1     p02700    (1, 362)     (1, 503)
2     p02700    (1, 361)     (1, 110)
3     p02700    (1, 362)     (1, 815)
4     p02700    (1, 362)    (1, 1554)
2 evaluate_val.ftr


...

  problem_id input_shape output_shape
0     p03916    (1, 651)    (1, 1100)
1     p03916    (1, 651)    (1, 1580)
2     p03916    (1, 651)     (1, 960)
3     p03916    (1, 651)     (1, 576)
4     p03916    (1, 651)     (1, 382)
7 reward_train.ftr


100%|██████████| 666324/666324 [59:12<00:00, 187.55it/s]  


  problem_id input_shape output_shape
0     p03644    (1, 305)      (1, 59)
1     p03644    (1, 305)      (1, 78)
2     p03644    (1, 305)     (1, 100)
3     p03644    (1, 305)     (1, 159)
4     p03644    (1, 305)     (1, 235)
8 reward_val.ftr


100%|██████████| 190379/190379 [18:02<00:00, 175.92it/s]


  problem_id input_shape output_shape
0     p03053    (1, 371)     (1, 999)
1     p03053    (1, 370)     (1, 393)
2     p03053    (1, 371)    (1, 1022)
3     p03053    (1, 370)     (1, 372)
4     p03053    (1, 370)    (1, 7815)


In [21]:
# get the max len of all tokenizations
# this cell takes around 20 minutes
maxlen_i = 0
maxlen_j = 0
for i, j in tqdm(cnds):
    bi, wi = i.input_ids.shape
    bj, wj = j.input_ids.shape

    if wi > maxlen_i:
        maxlen_i = wi

    if wj > maxlen_j:
        maxlen_j = wj

print(maxlen_i)
print(maxlen_j)

100%|██████████| 190378/190378 [18:58<00:00, 167.22it/s]  

1465
293127


In [51]:

dl = DataLoader(cnds, batch_size=10, shuffle=True)
for i, batch in enumerate(dl):
    print(i, batch[1].input_ids.shape)
    break

0 torch.Size([10, 1, 1028])
